In [16]:
!pip install matplotlib
!pip install sklearn

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
#convertir texto en un valor a una matrix, cada columna es una palabra
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import unicodedata
import pickle
import re
from sklearn.metrics import classification_report
from sklearn import metrics

In [18]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [19]:
df.sample(10)

,package_name,review,polarity
118,com.linkedin.android,totally diferent from the web page do not use...,0
807,org.mozilla.firefox,better background playback and 'new tab' posi...,1
322,com.viber.voip,why theres a delay on the messages. u will kn...,0
414,com.facebook.orca,we have wifi with full internet connection. ...,0
679,com.hamrokeyboard,hamro keyboard this is good app no doubt abou...,1
692,com.hamrokeyboard,need more modifications. undoubtly hamro keyb...,1
498,com.Slack,tablet support any chance of getting tablet s...,0
444,com.whatsapp,can't work after latest update i keep getting...,0
478,com.Slack,"edit: gif support, yay! if this can be addre...",0
516,com.dropbox.android,can't log into it tried to reset my password ...,0


In [20]:
#Eliminamos los caracteres desconocidos
df['review']=df['review'].str.strip()
df['review']=df['review'].str.lower()
df

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offlin...,0
1,com.facebook.katana,"messenger issues ever since the last update, i...",0
2,com.facebook.katana,profile any time my wife or anybody has more t...,0
3,com.facebook.katana,the new features suck for those of us who don'...,0
4,com.facebook.katana,forced reload on uploading pic on replying com...,0
...,...,...,...
886,com.rovio.angrybirds,loved it i loooooooooooooovvved it because it ...,1
887,com.rovio.angrybirds,all time legendary game the birthday party lev...,1
888,com.rovio.angrybirds,ads are way to heavy listen to the bad reviews...,0
889,com.rovio.angrybirds,fun works perfectly well. ads aren't as annoyi...,1


In [21]:
df['review']=df['review'].str.replace('!','')
df['review']=df['review'].str.replace(',','')
df['review']=df['review'].str.normalize('NFKC') #ANALIZA TEXTO NO LATINO Y LO TRATA DE ARREGLAR
df['review']=df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)
df.sample(10)

,package_name,review,polarity
163,com.imangi.templerun2,bolt i purchased bolt earlier in old phone. n...,0
119,com.king.candycrushsaga,why do u do this to me ok so i can't unlock th...,0
841,com.hamropatro,all nepali must use,1
689,com.hamrokeyboard,thanks for this app loved this app but my one ...,1
151,com.king.candycrushsaga,boosters gone last update took all my boosters...,1
280,com.tencent.mm,the translation are very hilarious the transla...,0
472,com.whatsapp,love it but there should be switch off or sign...,1
842,com.hamropatro,i love this app i love this app & its better t...,1
580,com.evernote,it's okay but i wish there's a way to archive ...,0
548,com.dropbox.android,between android or windows10my tablet or my ph...,1


In [22]:
def normalize_string(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else: 
        result=None
    return result

df['review']=df['review'].apply(normalize_string)
df.sample(10)

,package_name,review,polarity
370,com.google.android.talk,it was okay but now complete crap why do one n...,0
568,jabanaki.todo.todoly,only miss cache to be bought,0
691,com.hamrokeyboard,brilliant but not complete please add more emo...,1
527,com.dropbox.android,easy to use safe super easy to use fantastic o...,1
333,com.viber.voip,good app for call and texting. overall everyth...,0
765,com.shirantech.kantipur,too many pop up ads the frequent appearing pop...,0
128,com.king.candycrushsaga,umm.... while this game has gotten kinda old a...,1
863,com.rovio.angrybirds,angry birds what looks easy can demand a bit o...,1
425,com.facebook.orca,this has made my life hell it really sucks my ...,0
390,com.google.android.talk,mic on speaker is not working with oneplusx wo...,0


In [23]:
X=df['review']
y=df['polarity']

In [24]:
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=563, stratify=y)

In [25]:

#genera un vector, tfid lo pasa a una matriz, clf clasificacion con un binomial de bernoulli
text_clf = Pipeline([('vec',CountVectorizer(stop_words='english')),('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf.fit(X_train, y_train)

Pipeline(steps=[('vec', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [26]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

y_pred = text_clf.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average='weighted')

(0.8079532565668402, 0.7533632286995515, 0.7075961641787106, None)

In [27]:
print('Naive Bayes Train Accuracy = ',metrics.accuracy_score(y_train,text_clf.predict(X_train)))
print('Naive Bayes Test Accuracy = ',metrics.accuracy_score(y_test,text_clf.predict(X_test)))

Naive Bayes Train Accuracy =  0.8727544910179641
Naive Bayes Test Accuracy =  0.7533632286995515


In [28]:
n_iter_search = 5
parameters = {'vec__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
gs_clf = RandomizedSearchCV(text_clf, parameters, n_iter = n_iter_search)
gs_clf.fit(X_train, y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('vec',
                                              CountVectorizer(stop_words='english')),
                                             ('tfidf', TfidfTransformer()),
                                             ('clf', MultinomialNB())]),
                   n_iter=5,
                   param_distributions={'clf__alpha': (0.01, 0.001),
                                        'tfidf__use_idf': (True, False),
                                        'vec__ngram_range': [(1, 1), (1, 2)]})

In [29]:
gs_clf.best_params_

{'vec__ngram_range': (1, 1), 'tfidf__use_idf': False, 'clf__alpha': 0.01}

In [30]:
print('Naive Bayes Train Accuracy (grid random search) = ',metrics.accuracy_score(y_train,gs_clf.predict(X_train)))
print('Naive Bayes Test Accuracy (grid random search) = ',metrics.accuracy_score(y_test,gs_clf.predict(X_test)))

Naive Bayes Train Accuracy (grid random search) =  0.9880239520958084
Naive Bayes Test Accuracy (grid random search) =  0.8340807174887892


In [31]:
y_pred_mejor = gs_clf.predict(X_test)

print(classification_report(y_test, y_pred_mejor))

              precision    recall  f1-score   support

           0       0.82      0.95      0.88       146
           1       0.87      0.61      0.72        77

    accuracy                           0.83       223
   macro avg       0.85      0.78      0.80       223
weighted avg       0.84      0.83      0.83       223



In [32]:
best_model = gs_clf.best_estimator_
best_model

Pipeline(steps=[('vec', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf', MultinomialNB(alpha=0.01))])

In [33]:
# Save best model

pickle.dump(best_model, open('../models/best_model.pickle', 'wb'))

In [34]:
modelo = pickle.load(open('../models/best_model.pickle', 'rb')) # lo leemos
modelo.predict(X_test) # lo usamos para predecir nueva X_test

array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1])